#Importing libraries

In [ ]:
from __future__ import print_function
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from IPython.display import display, HTML

import sklearn as skl
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn import model_selection

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

#file management
import glob
import os

Setting up some constants

In [ ]:
#Some styling
pd.options.display.float_format = '{:.1f}'.format
sns.set()
plt.style.use('ggplot')

#Label encoder used to get a numeric representation of a label
le = preprocessing.LabelEncoder()

#The activities
LABELS = ['standing',
          'walking-natural',
          'walking-rider',
          'trotting-natural',
          'trotting-rider',
          'running-natural',
          'running-rider',
          'jumping',
          'grazing',
          'eating',
          'head shake',
          'shaking',
          'scratch-biting',
          'rubbing',
          'fighting',
          'rolling',
          'scared']

#Sliding windows
TIME_PERIODS = 80
STEP_DISTANCE = 40

#Datasets
FILES = sorted(glob.glob('horse_data/*.csv'))

#Set up dataframe

In [ ]:
REMOVE_COLUMNS = ['Mx', 'My', 'Mz','A3D','G3D','M3D'] #Add columns to drop from dataframe

def loadDataFrame(files):
    """
    Simple function to set up dataframe and initial clean-up of the data
    files: path to files
    returns: combined dataframe of all files
    """
    df = pd.concat((pd.read_csv(file) for file in files))
    df.drop(REMOVE_COLUMNS, axis=1, inplace=True)
    df['ActivityEncoded'] = le.fit_transform(df['label'].values.ravel())
    return df

def convert_to_float(x):

    try:
        return np.float(x)
    except:
        return np.nan


df = loadDataFrame(FILES)
df.head(10)

#Plot data composition

In [ ]:
#Training examples per activity type
df['label'].value_counts().plot(kind='bar', title='Training Examples of subject Viva by Activity Type')
plt.show()

SECONDS = 10 #nr of seconds to display accelerometer data
SAMPLING_RATE = 20 #the sampling rate at which data was recorded

def plot_activity(activity, data):

    fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(15, 10), sharex=True)
    plot_axis(ax0, data['datetime'], data['Ax'], 'X-Axis')
    plot_axis(ax1, data['datetime'], data['Ay'], 'Y-Axis')
    plot_axis(ax2, data['datetime'], data['Az'], 'Z-Axis')
    plt.subplots_adjust(hspace=0.2)
    fig.suptitle(activity)
    plt.subplots_adjust(top=0.90)
    plt.show()

def plot_axis(ax, x, y, title):

    ax.plot(x, y, 'r')
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)

#plot all 3 subplots for each activity
for activity in np.unique(df['label']):
    subset = df[df['label'] == activity][:SECONDS*SAMPLING_RATE] 
    plot_activity(activity, subset)


#Split training and test set

In [ ]:
#Shuffles and splits the data into a training set of 80% of the data and a test set containing the other 20%
train, test = skl.model_selection.train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

#Apply feature scaling

In [ ]:
#divide all 3 axis with the max value in the training set
train['Ax'] = train['Ax'] / train['Ax'].max()
train['Ay'] = train['Ay'] / train['Ay'].max()
train['Az'] = train['Az'] / train['Az'].max()

#divide all 3 axis with the max value in the training set
test['Ax'] = train['Ax'] / train['Ax'].max()
test['Ay'] = train['Ay'] / train['Ay'].max()
test['Az'] = train['Az'] / train['Az'].max()



# Round numbers up to 5 decimal spaces
train = train.round({'Ax': 5, 'Ay': 5, 'Az': 5})
test = test.round({'Ax': 5, 'Ay': 5, 'Az': 5})

#Segmentation of data (Sliding windows)

In [ ]:
LABEL = 'ActivityEncoded'

def create_segments_and_labels(df, time_steps, step, label_name):

    # x, y, z acceleration as features
    N_FEATURES = 3
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['Ax'].values[i: i + time_steps]
        ys = df['Ay'].values[i: i + time_steps]
        zs = df['Az'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        label = stats.mode(df[label_name][i: i + time_steps])[0][0]
        segments.append([xs, ys, zs])
        labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels

x_train, y_train = create_segments_and_labels(train, TIME_PERIODS, STEP_DISTANCE, LABEL)
x_test, y_test = create_segments_and_labels(test, TIME_PERIODS, STEP_DISTANCE, LABEL)

#set dimension of input and output
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
num_classes = le.classes_.size

#Flatten data to one dimension

In [ ]:
input_shape = (num_time_periods*num_sensors)
x_train = x_train.reshape(x_train.shape[0], input_shape)
x_test = x_test.reshape(x_test.shape[0], input_shape)

# Apply One-Hot encoding

In [ ]:
x_train = x_train.astype('float32')
y_train = y_train.astype('float32')
x_test = x_test.astype('float32')
y_test = y_test.astype('float32')
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)